In [17]:
import re

def extract_emails_and_phones(input_string):
    # Регулярное выражение для поиска email-адресов
    phone_pattern = r'\b(?:\+?\d{1,3}[-.\s]?)?(?:\(?\d{1,4}?\)?[-.\s]?)?\d{1,4}[-.\s]?\d{1,4}[-.\s]?\d{1,9}\b'
    email_pattern = r'[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}'

    
    # Регулярное выражение для поиска телефонных номеров
    #phone_pattern = r'\\b(?:\\+?\\d{1,3}[-.\\s]?)?(?:\\(?\\d{1,4}?\\)?[-.\\s]?)?\\d{1,4}[-.\\s]?\\d{1,4}[-.\\s]?\\d{1,9}\\b'
    
    # Поиск email-адресов
    emails = re.findall(email_pattern, input_string)
    
    # Поиск телефонных номеров
    phones = re.findall(phone_pattern, input_string)
    
    return emails, phones

# Пример использования
input_string = "Контактные данные: john.doe@example.com, 123-456-7890 и jane_doe123@gmail.com."
emails, phones = extract_emails_and_phones(input_string)
print("Электронные адреса:", emails)
print("Номера телефонов:", phones)

Электронные адреса: ['john.doe@example.com', 'jane_doe123@gmail.com']
Номера телефонов: ['123-456-7890']


In [18]:
def get_mail(mail_phone):
    return mail_phone[0]
def get_phone(mail_phone):
    return mail_phone[1]

In [19]:
def process_phone(phone):
 # Убираем +7 и 8, заменяем на 98
 #phone = phone.replace('+7', '98').replace('8', '98', 1)
 # Убираем скобки, тире и пробелы
 if phone == ' ' or phone == '':
     return np.nan
 phone = str(phone).replace('(', '').replace(')', '').replace('-', '').replace(' ', '').replace('+', '')
 phone = phone[1:]
 phone = "98" + phone
 # Убираем добавочные номера, если они есть
 phone = phone.split('доб.')[0] # если есть "доп." то берем только часть до него
 if phone == '98an' or len(phone) != 12:
     return np.nan
 return phone

In [20]:
def add_phones_to_comments(row):
    # Находим все телефоны с тем же названием
    phones = df[df['ИНН'] == row['ИНН']]['Основной телефон'].tolist()
    phones = [str(phone) for phone in phones]
    # Объединяем их в строку через запятую
    phones_str = ', '.join(phones[1:])
    # Добавляем телефоны к существующему комментарию
    return f"{row['Комментарий']} (Телефоны с других филиалов: {phones_str})"

def add_mails_to_comments(row):
    # Находим все телефоны с тем же названием
    mails = df[df['ИНН'] == row['ИНН']]['E-mail'].tolist()
    mails = [str(mail) for mail in mails]
    # Объединяем их в строку через запятую
    mails_str = ', '.join(phones[1:])
    # Добавляем телефоны к существующему комментарию
    return f"{row['Комментарий']} (Почты с других филиалов: {mails_str})"

In [21]:
from selenium import webdriver
import time
from datetime import datetime
import re
import requests
import random
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup as BS
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
import numpy as np
import pandas as pd
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [22]:
df = pd.read_excel(r"\\pecom.local\UserData\WKS_FR_DCR\ermakov.dst\Desktop\Рабочая папка\WB\Седьмая партия\спарк+яндекс3.xlsx")
df

,Unnamed: 0.1,ИНН,Ссылка на карточку,Название,Фокусная отрасль(запрос),Количество товаров продано,Официальное название,region,status,Контакты,Unnamed: 0,Почты,Контакты СПАРК и яндекс
0,0,561507269404,https://www.wildberries.ru/seller/455568,MaryashA,Фэшен,149 268,NaN,Московская область,Действующее,CHUMAKOVA_VA@MAIL.RU,NaN,NaN,"CHUMAKOVA_VA@MAIL.RU,"
1,1,665821766737,https://www.wildberries.ru/seller/4234077,B-Store,Фэшен,70 652,NaN,Свердловская область,Действующее,MALLAEV.AMIR@BK.RU,NaN,NaN,"MALLAEV.AMIR@BK.RU,"
2,2,290209522869,https://www.wildberries.ru/seller/3945406,Brenda Fashion,Фэшен,8 964,NaN,Архангельская область,Действующее,DMITRY_AKSENOV@HOTMAIL.COM,NaN,NaN,"DMITRY_AKSENOV@HOTMAIL.COM,"
3,3,22301565032,https://www.wildberries.ru/seller/272815,SG fashion & accessory,Фэшен,361 529,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,773001860640,https://www.wildberries.ru/seller/78504,B.B.Fashion,Фэшен,33 674,NaN,Москва,Действующее,SMART-HUNTER@YANDEX.RU,NaN,NaN,"SMART-HUNTER@YANDEX.RU,"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4230,4230,?,https://www.wildberries.ru/seller/403701,Парфюмерия By Shaik By,Парфюмерия,41 114,Индивидуальный предприниматель Нехвядович Арту...,NaN,NaN,NaN,84.0,nata_vystavkina@mail.ru,nata_vystavkina@mail.ru
4231,4231,?,https://www.wildberries.ru/seller/403701,Парфюмерия By Shaik By,Парфюмерия,41 114,Индивидуальный предприниматель Нехвядович Арту...,NaN,NaN,NaN,155.0,asv-tm@yandex.ru,asv-tm@yandex.ru
4232,4232,?,https://www.wildberries.ru/seller/403701,Парфюмерия By Shaik By,Парфюмерия,41 114,Индивидуальный предприниматель Нехвядович Арту...,NaN,NaN,NaN,259.0,info@region-a.kz,info@region-a.kz
4233,4233,?,https://www.wildberries.ru/seller/403701,Парфюмерия By Shaik By,Парфюмерия,41 114,Индивидуальный предприниматель Нехвядович Арту...,NaN,NaN,NaN,423.0,ankai3366@163.com,ankai3366@163.com


In [25]:
df = pd.read_excel(r"\\pecom.local\UserData\WKS_FR_DCR\ermakov.dst\Desktop\Рабочая папка\WB\Седьмая партия\спарк+яндекс7.xlsx")
df['Контакты'] = df['Контакты СПАРК и яндекс'].apply(extract_emails_and_phones)
df['Почты'] = df['Контакты'].apply(lambda x: x[0] if x else None)
df['Телефоны'] = df['Контакты'].apply(lambda x: x[1] if x else None)
df.insert(0, 'Основной телефон', df['Телефоны'].apply(lambda x: x[0] if x else None))
df.insert(2, 'E-mail', df['Почты'].apply(lambda x: x[0] if x else None))
df.drop_duplicates(subset = ['Основной телефон'])
df.drop_duplicates(subset = ['E-mail'])
df.reset_index(inplace=True, drop=True)
df.to_excel(r"\\pecom.local\UserData\WKS_FR_DCR\ermakov.dst\Desktop\Рабочая папка\WB\Обработка.xlsx")
df

,Основной телефон,Unnamed: 0.2,E-mail,Unnamed: 0.1,ИНН,Ссылка на карточку,Название,Фокусная отрасль(запрос),Количество товаров продано,Официальное название,region,status,Контакты,Unnamed: 0,Почты,Контакты СПАРК и яндекс,Телефоны
0,None,0,CHUMAKOVA_VA@MAIL.RU,0,561507269404,https://www.wildberries.ru/seller/455568,MaryashA,Фэшен,149 268,NaN,Московская область,Действующее,"([CHUMAKOVA_VA@MAIL.RU], [])",NaN,[CHUMAKOVA_VA@MAIL.RU],"CHUMAKOVA_VA@MAIL.RU,",[]
1,None,1,MALLAEV.AMIR@BK.RU,1,665821766737,https://www.wildberries.ru/seller/4234077,B-Store,Фэшен,70 652,NaN,Свердловская область,Действующее,"([MALLAEV.AMIR@BK.RU], [])",NaN,[MALLAEV.AMIR@BK.RU],"MALLAEV.AMIR@BK.RU,",[]
2,None,2,DMITRY_AKSENOV@HOTMAIL.COM,2,290209522869,https://www.wildberries.ru/seller/3945406,Brenda Fashion,Фэшен,8 964,NaN,Архангельская область,Действующее,"([DMITRY_AKSENOV@HOTMAIL.COM], [])",NaN,[DMITRY_AKSENOV@HOTMAIL.COM],"DMITRY_AKSENOV@HOTMAIL.COM,",[]
3,None,4,SMART-HUNTER@YANDEX.RU,4,773001860640,https://www.wildberries.ru/seller/78504,B.B.Fashion,Фэшен,33 674,NaN,Москва,Действующее,"([SMART-HUNTER@YANDEX.RU], [])",NaN,[SMART-HUNTER@YANDEX.RU],"SMART-HUNTER@YANDEX.RU,",[]
4,None,5,COKOLOVCKI@GMAIL.COM,5,771577539871,https://www.wildberries.ru/seller/55034,XELA,Фэшен,507 689,NaN,Мурманская область,Действующее,"([COKOLOVCKI@GMAIL.COM], [])",NaN,[COKOLOVCKI@GMAIL.COM],"COKOLOVCKI@GMAIL.COM,",[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2084,None,4213,SOLTANOV_RASIM@INBOX.RU,4213,231136188300,https://www.wildberries.ru/seller/669142,SHAIKH PARFUM,Парфюмерия,93 668,Индивидуальный предприниматель Оруджова Шабнам...,Волгоградская область,Действующее,"([SOLTANOV_RASIM@INBOX.RU], [])",NaN,[SOLTANOV_RASIM@INBOX.RU],"SOLTANOV_RASIM@INBOX.RU,",[]
2085,None,4216,ANDRIYANOVA_93@INBOX.RU,4216,581304781867,https://www.wildberries.ru/seller/183599,MARKETNA,Парфюмерия,22 365,Индивидуальный предприниматель Лебедева Натали...,Московская область,Действующее,"([ANDRIYANOVA_93@INBOX.RU], [])",NaN,[ANDRIYANOVA_93@INBOX.RU],"ANDRIYANOVA_93@INBOX.RU,",[]
2086,None,4219,AINAGUL-78@MAIL.RU,4219,551405483106,https://www.wildberries.ru/seller/1420874,ROSE BREND,Парфюмерия,22 826,ИП,Москва,Действующее,"([AINAGUL-78@MAIL.RU], [])",NaN,[AINAGUL-78@MAIL.RU],"AINAGUL-78@MAIL.RU,",[]
2087,None,4228,TEENKA@YANDEX.RU,4228,622809640920,https://www.wildberries.ru/seller/624327,Арома Мэджик,Парфюмерия,77 844,Индивидуальный предприниматель Богословская Лю...,Москва,Действующее,"([TEENKA@YANDEX.RU], [])",NaN,[TEENKA@YANDEX.RU],"TEENKA@YANDEX.RU,",[]


In [30]:
df = pd.read_excel(r"\\pecom.local\UserData\WKS_FR_DCR\ermakov.dst\Desktop\Рабочая папка\WB\Обработка.xlsx")
df.insert(2, 'Бизнес-юнит', "Сборный груз")
df.insert(3, 'Страна', "Россия")
#df.insert(4, 'ИНН', np.nan)
df.insert(7, 'ФИО контакта', np.nan)
#!!!!!!!!!!!!!!!!!!!!!!!data.insert(7, 'Основной город', data['Запрос'].apply(lambda x: x.split(" ")[0]))
df.insert(8, 'Ответственный в лиде (ФИО полностью)', np.nan)
df.insert(9, 'Ответственный за продажу (ФИО полностью)', np.nan)
df.insert(10, 'Источник', 'WB')
df.insert(11, 'Тег', 'Парсинг WB')
df.insert(13, 'Подразделение, выполняющее импорт', 'Департамент маркетинга')
df.insert(14, 'Комментарий', 'Название: ' + df['Название'] + '. Фокусная отрасль(запрос): ' +  df['Фокусная отрасль(запрос)'] + '. Количество товаров продано: ' +  df['Количество товаров продано'] + '. Регион: ' + df['region'] + '. Другие телефоны: ' + df['Телефоны'].apply(lambda x: ", ".join(str(x).split(", ")[1:])) + '. Другие почты: ' + df['Почты'].apply(lambda x: ", ".join(x.split(", ")[1:])))
df['Комментарий'] = df.apply(add_phones_to_comments, axis=1)
df['Комментарий'] = df.apply(add_mails_to_comments, axis=1)
#df['Комментарий'] = df['Комментарий'] + 'Официальное название: ' + df['Название'].fillna("")
df['Комментарий'] = df['Комментарий'] + 'Ссылка на карточку: ' + df['Ссылка на карточку']
df['Основной телефон'] = df['Основной телефон'].apply(process_phone)
df = df.drop_duplicates(subset = ['ИНН'])
df['Название'] = df['Официальное название']
df = df[df['region'] == 'Москва']
df = df.drop(['Unnamed: 0.3', 'Unnamed: 0.2', 'Ссылка на карточку', 'Unnamed: 0.1', 'Фокусная отрасль(запрос)', 'Количество товаров продано', 'Телефоны', 'region', 'status', 'Контакты', 'Unnamed: 0', 'Почты', 'Контакты СПАРК и яндекс'], axis=1)
df = df.reindex(columns = ['Основной телефон', 'Бизнес-юнит', 'Страна', 'ИНН', 'Название', 'ФИО Контакта', 'Основной город', 'Ответственный в лиде (ФИО полностью)', 'Ответственный за продажу (ФИО полностью)', 'Источник', 'Тег', 'E-mail', 'Подразделение, выполняющее импорт', 'Комментарий'])
#df = df.drop_duplicates(subset = ['Основной телефон'])
df = df[(df['ИНН'].str.len() == 10) | (df['ИНН'].str.len() == 12)]
df['Основной город'] = 'Москва'
df.to_excel(r"\\pecom.local\UserData\WKS_FR_DCR\ermakov.dst\Desktop\Рабочая папка\WB\Обработка3.xlsx")
df

,Основной телефон,Бизнес-юнит,Страна,ИНН,Название,ФИО Контакта,Основной город,Ответственный в лиде (ФИО полностью),Ответственный за продажу (ФИО полностью),Источник,Тег,E-mail,"Подразделение, выполняющее импорт",Комментарий
3,NaN,Сборный груз,Россия,773001860640,NaN,NaN,Москва,NaN,NaN,WB,Парсинг WB,SMART-HUNTER@YANDEX.RU,Департамент маркетинга,Название: B.B.Fashion. Фокусная отрасль(запрос...
12,NaN,Сборный груз,Россия,672505980701,NaN,NaN,Москва,NaN,NaN,WB,Парсинг WB,SF@RESTOTOUCH.RU,Департамент маркетинга,Название: Lion Group. Фокусная отрасль(запрос)...
13,NaN,Сборный груз,Россия,772377986232,NaN,NaN,Москва,NaN,NaN,WB,Парсинг WB,USTINOW@MAIL.RU,Департамент маркетинга,Название: CRASAVA. Фокусная отрасль(запрос): Ф...
15,NaN,Сборный груз,Россия,532007579591,NaN,NaN,Москва,NaN,NaN,WB,Парсинг WB,I@DONAKEN.RU,Департамент маркетинга,Название: Женская одежда DONAT. Фокусная отрас...
20,NaN,Сборный груз,Россия,7721793895,NaN,NaN,Москва,NaN,NaN,WB,Парсинг WB,3206320@mail.ru,Департамент маркетинга,Название: КОМУС. Фокусная отрасль(запрос): Фэш...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2079,NaN,Сборный груз,Россия,772002682814,Индивидуальный предприниматель Гоним Анна Алек...,NaN,Москва,NaN,NaN,WB,Парсинг WB,ANNA.AT@MAIL.RU,Департамент маркетинга,Название: Вишневый джем. Фокусная отрасль(запр...
2081,NaN,Сборный груз,Россия,9721215506,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""АМАНАТ""",NaN,Москва,NaN,NaN,WB,Парсинг WB,salamat_17@mail.ru,Департамент маркетинга,Название: Branded perfumes. Фокусная отрасль(з...
2083,984997887130,Сборный груз,Россия,7727194489,ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ''ПАР...,NaN,Москва,NaN,NaN,WB,Парсинг WB,NaN,Департамент маркетинга,Название: ПАРФЮМЕРИЯ XXI ВЕКА. Фокусная отрасл...
2086,NaN,Сборный груз,Россия,551405483106,ИП,NaN,Москва,NaN,NaN,WB,Парсинг WB,AINAGUL-78@MAIL.RU,Департамент маркетинга,Название: ROSE BREND. Фокусная отрасль(запрос)...


In [31]:
invalid_inn_count = df[(df['ИНН'].str.len() != 10) & (df['ИНН'].str.len() != 12)]
df = df[(df['ИНН'].str.len() == 10) | (df['ИНН'].str.len() == 12)]
invalid_inn_count
df = df[(df['ИНН'].str.len() == 10) | (df['ИНН'].str.len() == 12)]
df

,Основной телефон,Бизнес-юнит,Страна,ИНН,Название,ФИО Контакта,Основной город,Ответственный в лиде (ФИО полностью),Ответственный за продажу (ФИО полностью),Источник,Тег,E-mail,"Подразделение, выполняющее импорт",Комментарий
3,NaN,Сборный груз,Россия,773001860640,NaN,NaN,Москва,NaN,NaN,WB,Парсинг WB,SMART-HUNTER@YANDEX.RU,Департамент маркетинга,Название: B.B.Fashion. Фокусная отрасль(запрос...
12,NaN,Сборный груз,Россия,672505980701,NaN,NaN,Москва,NaN,NaN,WB,Парсинг WB,SF@RESTOTOUCH.RU,Департамент маркетинга,Название: Lion Group. Фокусная отрасль(запрос)...
13,NaN,Сборный груз,Россия,772377986232,NaN,NaN,Москва,NaN,NaN,WB,Парсинг WB,USTINOW@MAIL.RU,Департамент маркетинга,Название: CRASAVA. Фокусная отрасль(запрос): Ф...
15,NaN,Сборный груз,Россия,532007579591,NaN,NaN,Москва,NaN,NaN,WB,Парсинг WB,I@DONAKEN.RU,Департамент маркетинга,Название: Женская одежда DONAT. Фокусная отрас...
20,NaN,Сборный груз,Россия,7721793895,NaN,NaN,Москва,NaN,NaN,WB,Парсинг WB,3206320@mail.ru,Департамент маркетинга,Название: КОМУС. Фокусная отрасль(запрос): Фэш...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2079,NaN,Сборный груз,Россия,772002682814,Индивидуальный предприниматель Гоним Анна Алек...,NaN,Москва,NaN,NaN,WB,Парсинг WB,ANNA.AT@MAIL.RU,Департамент маркетинга,Название: Вишневый джем. Фокусная отрасль(запр...
2081,NaN,Сборный груз,Россия,9721215506,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""АМАНАТ""",NaN,Москва,NaN,NaN,WB,Парсинг WB,salamat_17@mail.ru,Департамент маркетинга,Название: Branded perfumes. Фокусная отрасль(з...
2083,984997887130,Сборный груз,Россия,7727194489,ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ''ПАР...,NaN,Москва,NaN,NaN,WB,Парсинг WB,NaN,Департамент маркетинга,Название: ПАРФЮМЕРИЯ XXI ВЕКА. Фокусная отрасл...
2086,NaN,Сборный груз,Россия,551405483106,ИП,NaN,Москва,NaN,NaN,WB,Парсинг WB,AINAGUL-78@MAIL.RU,Департамент маркетинга,Название: ROSE BREND. Фокусная отрасль(запрос)...


In [32]:
#df = df.drop_duplicates(subset = ['Основной телефон'])
#df = df.drop_duplicates(subset = ['Название'])
df['Ответственный в лиде (ФИО полностью)'].iloc[0:420:5] = 'Баркова Виктория Юрьевна'
df['Ответственный в лиде (ФИО полностью)'].iloc[1:420:5] = 'Дементьев Даниил Сергеевич'
df['Ответственный в лиде (ФИО полностью)'].iloc[2:420:5] = 'Переверзева Мария Валерьевна'
df['Ответственный в лиде (ФИО полностью)'].iloc[3:420:5] = 'Прозоров Максим Андреевич'
df['Ответственный в лиде (ФИО полностью)'].iloc[4:420:5] = 'Савицкая Нила Андреевна'
df.to_excel(r"\\pecom.local\UserData\WKS_FR_DCR\ermakov.dst\Desktop\Рабочая папка\WB\Обработка4.xlsx")
df

C:\Users\ermakov.dst\AppData\Local\Temp\ipykernel_12012\1731243034.py:3: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df['Ответственный в лиде (ФИО полностью)'].iloc[0:420:5] = 'Баркова Виктория Юрьевна'
C:\Users\ermakov.dst\AppData\Local\T

,Основной телефон,Бизнес-юнит,Страна,ИНН,Название,ФИО Контакта,Основной город,Ответственный в лиде (ФИО полностью),Ответственный за продажу (ФИО полностью),Источник,Тег,E-mail,"Подразделение, выполняющее импорт",Комментарий
3,NaN,Сборный груз,Россия,773001860640,NaN,NaN,Москва,Баркова Виктория Юрьевна,NaN,WB,Парсинг WB,SMART-HUNTER@YANDEX.RU,Департамент маркетинга,Название: B.B.Fashion. Фокусная отрасль(запрос...
36,984992016383,Сборный груз,Россия,7717730846,NaN,NaN,Москва,Дементьев Даниил Сергеевич,NaN,WB,Парсинг WB,n.silanteva@detprod.ru,Департамент маркетинга,Название: ДЕТСКИЕ ПРОДУКТЫ. Фокусная отрасль(з...
37,984951388855,Сборный груз,Россия,9704137431,NaN,NaN,Москва,Переверзева Мария Валерьевна,NaN,WB,Парсинг WB,NaN,Департамент маркетинга,Название: Гранд-Трейд. Фокусная отрасль(запрос...
53,984957417301,Сборный груз,Россия,7728639290,NaN,NaN,Москва,Прозоров Максим Андреевич,NaN,WB,Парсинг WB,info@peligrin.ru,Департамент маркетинга,Название: Пелигрин Матен. Фокусная отрасль(зап...
62,984957504884,Сборный груз,Россия,7734610210,NaN,NaN,Москва,Савицкая Нила Андреевна,NaN,WB,Парсинг WB,info@daisy.ru,Департамент маркетинга,Название: DAISY. Фокусная отрасль(запрос): Пел...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1987,984955092418,Сборный груз,Россия,7714560730,"Общество с ограниченной ответственностью ""В2-П...",NaN,Москва,Баркова Виктория Юрьевна,NaN,WB,Парсинг WB,NaN,Департамент маркетинга,Название: Парфюмерная компания Neo Parfum. Фок...
2027,984956400852,Сборный груз,Россия,7713408395,ГИПАРЛИЗ ПАРФЮМ ООО,NaN,Москва,Дементьев Даниил Сергеевич,NaN,WB,Парсинг WB,info@contract-cosmetica.ru,Департамент маркетинга,Название: Парфюмерный Рай. Фокусная отрасль(за...
2033,984957898711,Сборный груз,Россия,5032262632,"Общество с ограниченной ответственностью ""Понт...",NaN,Москва,Переверзева Мария Валерьевна,NaN,WB,Парсинг WB,p1@pontiparfum.ru,Департамент маркетинга,Название: Понти Парфюм. Фокусная отрасль(запро...
2047,989035875592,Сборный груз,Россия,7743247699,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ТД ""Э...",NaN,Москва,Прозоров Максим Андреевич,NaN,WB,Парсинг WB,cosmoskidka@yandex.ru,Департамент маркетинга,Название: EXCLUSIVE Shop. Фокусная отрасль(зап...
